# Task 4: FICO Scores
Charlie wants to make her model work for future data sets, so she needs a general approach to generating the buckets. Given a set number of buckets corresponding to the number of input labels for the model, she would like to find out the boundaries that best summarize the data. You need to create a rating map that maps the FICO score of the borrowers to a rating where a lower rating signifies a better credit score.

The process of doing this is known as quantization. You could consider many ways of solving the problem by optimizing different properties of the resulting buckets, such as the mean squared error or log-likelihood (see below for definitions). For background on quantization, see here.

Mean squared error

You can view this question as an approximation problem and try to map all the entries in a bucket to one value, minimizing the associated squared error. We are now looking to minimize the following:

Log-likelihood

A more sophisticated possibility is to maximize the following log-likelihood function:

Where bi is the bucket boundaries, ni is the number of records in each bucket, ki is the number of defaults in each bucket, and pi = ki / ni is the probability of default in the bucket. This function considers how rough the discretization is and the density of defaults in each bucket. This problem could be addressed by splitting it into subproblems, which can be solved incrementally (i.e., through a dynamic programming approach). For example, you can break the problem into two subproblems, creating five buckets for FICO scores ranging from 0 to 600 and five buckets for FICO scores ranging from 600 to 850. Refer to this page for more context behind a likelihood function. This page may also be helpful for background on dynamic programming. 



In [1]:
# Step 1: Load Data
import pandas as pd
import numpy as np

df = pd.read_csv('Task 3 and 4_Loan_Data.csv')
x = df['default'].to_list()        # binary labels: 1 if defaulted, 0 if not
y = df['fico_score'].to_list()     # FICO scores
n = len(x)

print("Length of x:", len(x))
print("Length of y:", len(y))

Length of x: 10000
Length of y: 10000


### Step 2: Count defaults and totals for each FICO score
We build two arrays:
- `default[i]` = number of defaults at FICO = i + 300
- `total[i]` = total number of people at FICO = i + 300

This assumes FICO scores range from 300 to 850 (inclusive), giving us 551 possible values.

In [2]:
default = [0 for _ in range(551)]
total = [0 for _ in range(551)]

for i in range(n):
    score = int(y[i]) - 300
    default[score] += x[i]  # x[i] is 1 or 0
    total[score] += 1

### Step 3: Compute prefix sums
We compute cumulative sums of defaults and totals for efficient range queries later.

In [3]:
for i in range(1, 551):
    default[i] += default[i - 1]
    total[i] += total[i - 1]

print("defaults: ", default)
print("totals:, ", total)

defaults:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 9, 13, 13, 13, 13, 13, 14, 17, 18, 19, 21, 23, 24, 27, 29, 30, 31, 32, 33, 35, 36, 37, 38, 38, 40, 42, 46, 48, 52, 53, 57, 59, 63, 65, 67, 73, 74, 81, 81, 89, 91, 93, 97, 102, 106, 112, 117, 121, 124, 127, 131, 135, 137, 142, 146, 149, 154, 160, 162, 166, 171, 175, 180, 187, 192, 199, 206, 211, 213, 217, 218, 220, 230, 238, 243, 248, 255, 263, 269, 277, 283, 295, 300, 307, 312, 317, 328, 333, 341, 347, 357, 369, 375, 384, 397, 411, 418, 428, 433, 445, 453, 459, 470, 486, 494, 499, 509, 516, 535, 543, 553, 566, 581, 59

### Step 4: Define log-likelihood function for a binomial distribution
We use the binomial log-likelihood of observing `k` defaults in `n` samples with empirical probability `p = k / n`.

In [4]:
def log_likelihood(n, k):
    p = k / n
    if p == 0 or p == 1:
        return 0
    return k * np.log(p) + (n - k) * np.log(1 - p)

### Step 5: Dynamic Programming Initialization
We define a DP table:
- `dp[i][j][0]` = max log-likelihood for `i` buckets ending at index `j`
- `dp[i][j][1]` = backtracking pointer to the previous split

We initialize the DP table with very negative values to simulate `-inf`.

In [5]:
r = 10  # Number of desired buckets
dp = [[[-10 ** 18, 0] for _ in range(551)] for _ in range(r + 1)]

for i in range(r + 1):
    for j in range(551):
        if i == 0:
            dp[i][j][0] = 0
        else:
            for k in range(j):
                if total[j] == total[k]:
                    continue
                ll = log_likelihood(total[j] - total[k], default[j] - default[k])
                if i == 1:
                    dp[i][j][0] = ll
                else:
                    if dp[i][j][0] < dp[i - 1][k][0] + ll:
                        dp[i][j][0] = dp[i - 1][k][0] + ll
                        dp[i][j][1] = k

### Step 6: Extract optimal result
We extract the final log-likelihood and backtrack to recover the optimal bucket boundaries.

In [6]:
result = round(dp[r][550][0], 4)
print("Maximum Log-Likelihood:", result)

# Recover bucket boundaries
k = 550
buckets = []
while r >= 0:
    buckets.append(k + 300)
    k = dp[r][k][1]
    r -= 1

print("Bucket Boundaries (FICO):", sorted(buckets))

Maximum Log-Likelihood: 0
Bucket Boundaries (FICO): [300, 812, 814, 816, 822, 823, 824, 825, 826, 828, 850]
